In [1]:
!pip install librosa tensorflow

Looking in indexes: https://mirrors.bfsu.edu.cn/pypi/web/simple
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl (253 kB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/ed/1a/b4ab4b8f8b3a41fade4899fd00b5b2d2dad0981f3e1bb10df4c522975fd7/tensorflow-2.15.0.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.3 MB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl (23 kB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/c1/07/7591f4efd29e65071c3a61b53725036ea8f73366a4920a481ebddaf8d0ca/soundfile-0.12.1-py2.py3-none-manylinux_2_31_x86_64.whl (1.2 MB)
  Using cached https://mirrors.bfsu.edu.cn/pypi/web/packages/1a/a5/5174dac3957ac412e80a00f30b6507031fcab7000afc9ea0ac413bddcff2/pooch-1.8.0-py3-none-any.whl (62 kB)
  Using cached htt

In [2]:
import librosa
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

2024-01-10 07:33:19.445204: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 07:33:19.445307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 07:33:19.447417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 07:33:19.462040: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 07:33:20.718493: W tensorflow/compiler/tf2

音频输入标准化处理为10S的片段

In [ ]:
audio, sr = librosa.load('audio_file.wav', sr=None)
# 指定每个片段的时长，单位为秒
duration = 10
segments = librosa.util.frame(audio, int(duration * sr), int(duration * sr))
for i, segment in enumerate(segments):
    librosa.output.write_wav(f'segment_{i}.wav', segment, sr)

音频提取MFCC特征

In [3]:
# 定义特征提取函数
max_pad_len = 432
np.set_printoptions(threshold=sys.maxsize)
def mfcc_feature(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    pad_width = np.abs(max_pad_len - mfccs.shape[1])
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs

In [13]:
# 提取MFCC特征的代码在这里执行，使用wav_filepath作为输入文件路径
mfcc = mfcc_feature('dataset/6dB/fan/id_00/abnormal/00000001.wav')
mfcc = np.asarray(mfcc, dtype=np.int32).reshape(mfcc.shape[0],mfcc.shape[1])
mfcc = mfcc.flatten()

In [14]:
# 数据恢复原始tensor模式
rows = 40
columns = 432
channels = 1

mfcc = mfcc.reshape(1,rows, columns, channels)

In [15]:
print(mfcc.shape)

(1, 40, 432, 1)


构建网络，并根据权重对输入推导

In [16]:
# Construct model 
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=2, input_shape=(rows, columns, channels), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=32, kernel_size=2, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=64, kernel_size=2, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(filters=128, kernel_size=2, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(1, activation='softmax')
])


In [17]:
# 编译模型
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [18]:
#导入权重
model.load_weights('mimii_weights.h5')

In [19]:
predictions = model.predict(mfcc)

1/1 [==============================] - 0s 98ms/step


In [20]:
#字符串替换
release = ['normal' if pred == 1 else 'abnormal' for pred in predictions]

In [21]:
print(release)

['normal']
